# 表示回路２テストプログラム
- DDR4に格納された画像を表示する
- PLにはDMA(AXIバスマスター側)と表示回路＋レジスタを実装
- VGAとXGAの２つの解像度の画像表示をテストする

### [STEP-1] 定数定義
- 表示回路２の解像度はVGA or XGA
- レジスタアドレス設定

In [ ]:
# 表示解像度パラメータ
PIXEL_VGA_WIDTH  = 640
PIXEL_VGA_HEIGHT = 480

PIXEL_XGA_WIDTH  = 1024
PIXEL_XGA_HEIGHT = 768

PIXEL_SXGA_WIDTH  = 1280
PIXEL_SXGA_HEIGHT = 1024

# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

### [STEP-2] DisplayPort設定用クラス定義
- Zynq内部の専用レジスタの定義とレジスタ設定

In [ ]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

### [STEP-3] Overlay
- 表示回路１のコンフィギュレーション

In [ ]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-4] パターン表示回路の解像度設定
- RESOLの値を変更してパターン表示回路の解像度を設定
    - 0 : VGA設定
    - 1 : XGA設定
    - 2 : SXGA設定


In [ ]:
RESOL = 1
pl.axi_gpio_0.write(RA_GPIO_DATA,(RESOL))
pl.axi_gpio_0.read(RA_GPIO_DATA)

### [STEP-5] DisplayPort初期化
- パターン表示回路の解像度に設定を合わせる
    - VGA設定  : dp.configure(VideoMode(640,480,24),PIXEL_RGB)
    - XGA設定  : dp.configure(VideoMode(1024,768,24),PIXEL_RGB)
    - SXGA設定 : dp.configure(VideoMode(1280,1024,24),PIXEL_RGB)

In [ ]:
from pynq.lib.video import *
lvdp = LiveDisplayPort()

if RESOL == 0:
    PIXEL_WIDTH  = PIXEL_VGA_WIDTH
    PIXEL_HEIGHT = PIXEL_VGA_HEIGHT
    print("VGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)
elif RESOL == 1 :
    PIXEL_WIDTH  = PIXEL_XGA_WIDTH
    PIXEL_HEIGHT = PIXEL_XGA_HEIGHT
    print("XGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)
else :
    PIXEL_WIDTH  = PIXEL_SXGA_WIDTH
    PIXEL_HEIGHT = PIXEL_SXGA_HEIGHT
    print("SXGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)
    

lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

### [STEP-6] VRAM領域確保
- １フレーム分の領域を確保
- 高さ x 幅 x 4byte(RGBA)で１フレーム

In [ ]:
from pynq import allocate

fb0 = allocate(shape = (PIXEL_HEIGHT, PIXEL_WIDTH, 4), dtype = "u1")

### [STEP-7] VRAM領域にテストパターン画像を書き込み
- パターン表示回路で表示した市松模様のパターン
- データの並びは、[B-8bit, G-8bit, R-8bit, A-8bit]の順
- ピクセル単位で書き込みをしているため、処理に数分かかります

In [ ]:
PIXEL_PATTERN = [[0,0,255,0],[0,255,0,0],[255,0,0,0],[255,0,255,0],[0,255,255,0],[255,255,0,0],[0,0,0,0],[255,255,255,0]]

for h in range(PIXEL_HEIGHT):
    for w in range(PIXEL_WIDTH):
        fb0[h][w][:] = PIXEL_PATTERN[w//(PIXEL_WIDTH>>3)-h//(PIXEL_HEIGHT>>2)]

### [STEP-8] 表示回路のレジスタ設定
- 同期処理（VBLANK待ち)
- 表示開始先頭アドレス設定 @ DISPADDR
- 表示ON @ DISPCTRL

In [ ]:
pl.regbus_0.write(RA_DISPCTRL, 0x00000003)
while pl.regbus_0.read(RA_DISPCTRL) != 0x00000003 :
    pass

pl.regbus_0.write(RA_DISPADDR, fb0.device_address)
pl.regbus_0.write(RA_DISPCTRL, 0x00000001)

### [STEP-9] 表示レジスタの設定
- 表示OFF @ DISPCTRL

In [ ]:
pl.regbus_0.write(RA_DISPCTRL, 0x00000000)

### [STEP-10] PYNQの初期画面に戻す
- DisplayPortの制御は、PL側からPS側に切り替わります。

In [ ]:
lvdp.close()